<a href="https://colab.research.google.com/github/mafux777/kintusgi-extras/blob/main/Kintsugi_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's start with some utilities for substrate for python..

In [1]:
# next line only needed in Google Colaboratory! Otherwise pip install yourself
!pip install substrate-interface

from substrateinterface import SubstrateInterface

kint_substrate = SubstrateInterface(
    url="wss://api-kusama.interlay.io/parachain"
)
ksm_substrate = SubstrateInterface(
    url="wss://kusama-rpc.dwellir.com"
)

def convert_kint_to_ksm(kint):
    universal = kint_substrate.ss58_decode(kint)
    kusama = ksm_substrate.ss58_encode(universal)
    return kusama

def convert_ksm_to_kint(k):
    universal = ksm_substrate.ss58_decode(k)
    kint = kint_substrate.ss58_encode(universal)
    return kint



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 182 kB 7.7 MB/s 
     |████████████████████████████████| 1.0 MB 62.1 MB/s 
     |████████████████████████████████| 1.0 MB 48.4 MB/s 
     |████████████████████████████████| 1.2 MB 54.7 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 
     |████████████████████████████████| 119 kB 47.2 MB/s 
     |████████████████████████████████| 2.3 MB 49.1 MB/s 
     |████████████████████████████████| 76 kB 1.5 MB/s 
     |████████████████████████████████| 243 kB 58.2 MB/s 
     |████████████████████████████████| 856 kB 69.0 MB/s 
     |████████████████████████████████| 481 kB 45.8 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
  Created wheel for cytoolz: filename=cytoolz-0.11.2-cp37-cp37m-linux_x86_64.whl size=1236692 sha256=e3275308b9ffef8d6febd0418854319e560ae32970b8945bac7900c577150446
  Stored in directory: /root/.cache/pip/wheels

Let's analyze Kintsugi's vaults and early investors. For that, we will use several subsquids.

Kintsugi-X is a subsquid built by the Kintsugi-X team for the purpose of analysing transfers.

In [2]:
import requests
import json
import pandas as pd

kintsugi = "https://api-kusama.interlay.io/graphql/graphql"
kusama = "https://app.gc.subsquid.io/beta/kusama-explorer/v1/graphql"
kintsugi_x = "https://app.gc.subsquid.io/beta/kintsugi-x/v4/graphql"


Version 4 of the Kintsugi-X squid ignored those token transfers of amount 456621004566 which are related to distributing vault block rewards. It's currently hard coded to the amount but it works.

One of the interesting questions is: do vaults self-mint?

In [3]:
def is_equal(row):
    return row["userParachainAddress"] == row['vault.accountId']


Let's start creating a directory of interesting Kintsugi addresses.
This info comes from the Interlay discord server, "meet the vaults".

In [4]:
kts = {
    "a3eZhSC12zE4D49ir4QkxZwDC3jU6iwNQ953ZZzQsCLTnPJjs": ['Simon Kraus', '@sik'],
    "a3fxS24Bduq8cxT4oV7RQ6q6vfEdSEkaapER75BS4rF7U6Q63": ['Simon Kraus', '@sik'],
    "a3addPTx9ngWGKq3dguw7vs7NA2PimcDUHWJ32HsuoFL74zdo": ['Simon Kraus', '@sik'],
    "a3bccyaV6tCoqaWqByeqvXo5kBbd1m4yCPKyX937HB1APTt7Y": ['rodrigo.barrios', 'hypersphere'],
    "a3bzFrZ5kXYpaaD5NbapUDSfjZPQTWFKGwSbMmGeRAL8BGrCs": ['@boyswan'],
    "a3btcmyVE6ENtWVyHiX9QnorJfKfA2TsSCF43urDeNAWKueH6": ['@seergeist'],
    "a3dh62XsvNmtPAzfwCH9bv34dqPFzjKcWBi5mYM93mKmMt64s": ['@DkA7s'],
    "a3dh7jXhw2q7vqUpEPAb8BeFTNqXXvG9zBf7hsfm3o9hNbjtK": ['@whisperit'],
    "a3aDPraojQvYhVHjyVuYRFXno58EPMjegrY9nubPmpck2X7JS": ['@blinkin', 'chaos DAO'],
    "a3azGTG3qGmUuQckCKjFAhfjfnnRAXmpgV4fPVPziNaA1zCwG": ['@marvel'],
    "a3eKvTxY56smUwHU9vLpw9w5kSqpoPkJskU2tNxUSHAnntQTS": ['@mafux777'],
    "a3cCyigH5pLJXcLKRNGFaBnx3a7diTXq9pPZ1TB8XWgqeCQvW": ['@spazcoin', 'chaos DAO', 'VaaS'],
    "a3baaLbC1JMHJLJ2HwEQMz3S5VuiCWBYy4i66Ziq1vXzmVU6b": ['@spazcoin', 'chaos DAO'],
    "a3fudELrRCjuSyYEPkRAKFQyjzo5YyU228LdqinGsnjBUNB8P": ['@spazcoin', 'chaos DAO', 'VaaS'],
    "a3cCyigH5pLJXcLKRNGFaBnx3a7diTXq9pPZ1TB8XWgqeCQvW": ['@spazcoin', 'chaos DAO'],
    "a3fcMNTjXcJSwAVnTNKwwP7T8XM2bCW7FshsTW2hpUTrdXzed": ['@spazcoin', 'chaos DAO'],
    "a3aPvmjypKaDtjRgYbDL2CCkseYR3SLwvPevL6j7wF67aFtV4": ['@timbotronic'],
    "a3fZSzXxTZYY58BQrfhJx8cDtp4wRdbZ8X4ReF2iUT63y5RcX": ['@0xvault'],
    "a3dJfVzssBJgBmRuMZBre5H71rvawHJoFvFHGz2Aq7Hdt492w": ['@niko'],
    "a3eFe9M2HbAgrQrShEDH2CEvXACtzLhSf4JGkwuT9SQ1EV4ti": ['@paride'],
    "a3cAyFZMgahPoAyWbNRrjX2TnXQtpS3bztCVMNuLNcTYATBte": ['@dan', 'interlay'],
    "a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe": ['pumpernickel'],
    "a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE": ['@alibaba'],
    "a3cDUVdQi8FqkiUBMjqS3RBWPghRWFXBwjRzKZvmB3MmHLDcP": ['@alibaba'],
    "a3aGT3FRF1WgWtWdi8VmhB3YEJVE6XtrvT41TqcKFxZbuUvS1": ['@warinelly'], # info provided by @quin
    "a3fxCiEXfZJThTpHR5FPT3jngnFWQygAtw1ieDp2w9669v6eN": ['@kbl', 'Kusama Kings & Queens'],
    "a3chQwNk9sxP4WHnm82cD6a5AR2tXJ7bmJ88yZGQp4Ay957PB": ['Lucky Friday', 'LIQUIDATED'],
    "a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3": ['Lucky Friday'],
    "a3fxCiEXfZJThTpHR5FPT3jngnFWQygAtw1ieDp2w9669v6eN": ['Lucky Friday'],
    "a3dCFNgcQsgL4wX6apeaxVLTyRzeCiUdEczyvPcXZ5Dmg5rez": ['Lucky Friday'],
    "a3aVQQpfHH49ACiLzDZP8w8eTA2oW7gvifX7E6dLsBJTBdzVR": ['Lucky Friday'],
}

In [5]:
def add_label_to_list(my_list, label):
  for a in my_list:
    if a not in kts:
      kts[a] = []
    if label not in kts[a]:
      kts[a].append(label)    
  
def enrich_df(df, col):
  df[f'label_{col}'] = df[col].apply(lambda v: "/".join(kts.get(v, [])))

Let's create a query to understand transfers from Kusama (Relay Chain) to Kintsugi. These have to be KSM transfers, because Kusama itself does not handle other assets.
We assume that all of these will end up as collateral. 


In [6]:
to_kintsugi = """
query MyQuery {
  transfers(where: {name_eq: "xcmPallet.reserveTransferAssets", AND: {to: {id_eq: "F7fq1inhrJsYSUkWhyZ3zqtp5K3AKBBjbPWy6VLiRGHipPi"}}}, orderBy: date_DESC) {
    name
    amount
    from {
      id
    }
    date
    to {
      id
    }
  }
}

"""

In [7]:
r = requests.post("https://app.gc.subsquid.io/beta/kusama-explorer/v1/graphql", json={"query" : to_kintsugi}).json()
df_0 = pd.json_normalize(r['data']['transfers'])
df_0['ksm'] = df_0['amount'].apply(lambda x: float(x) / 1e12)
kusama_transfers = df_0.groupby('from.id').agg(dict(ksm="sum", date="min")).sort_values('ksm', ascending=False)
kusama_transfers.reset_index(inplace=True)
kusama_transfers.rename(columns={"from.id": "kusama",
                                 "date": "first_seen"}, inplace=True)
kusama_transfers['kintsugi'] = kusama_transfers.kusama.apply(convert_ksm_to_kint)
kusama_transfers


,kusama,ksm,first_seen,kintsugi
0,GDN8YBtPVRkjR7YtEKMUMXzBdtsNYWW7djdsxcQDqSZSxmM,10000.00000,2022-04-25T15:53:42.020Z,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g
1,HExSfZUFJPkbUxpKQ5jM54TA7ThwfsCRBfgowmzzCcH8rV2,3446.75000,2022-05-02T15:20:06.029Z,a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3
2,CrZxfsquEZeXLZUk7Uxdts8NpKz1oc7pdDXwHYCSeJ8SLgW,2960.09492,2022-04-26T20:29:06.018Z,a3aVQQpfHH49ACiLzDZP8w8eTA2oW7gvifX7E6dLsBJTBdzVR
3,GcohCErDM7LjUfBkmNqu19CNzRUEp84DQP4nZeUhjrBHqNP,1067.96346,2022-04-25T17:29:30.016Z,a3eFe9M2HbAgrQrShEDH2CEvXACtzLhSf4JGkwuT9SQ1EV4ti
4,Ea9GXVeSWj1vxXutpMPtZAinuACMeWYyVfKHqKrn2gb11kj,993.73600,2022-04-26T07:15:42.025Z,a3cCyigH5pLJXcLKRNGFaBnx3a7diTXq9pPZ1TB8XWgqeCQvW
...,...,...,...,...
64,HcNCnh9ZLYRcXB7wm3aPyhzS4i3ghoKP6UR4intuJvp8buf,0.05000,2022-04-25T18:25:06.087Z,a3fFCewUawA7wHtxdRCwkhDVKDHBZnb7vDzN7E4bZdy5sKwfN
65,D8yhHcbwNuYEJUoSDwyQXVnGJzfGpEpnRq8Qdmkc1Etw8vE,0.05000,2022-04-25T19:23:30.004Z,a3amp9SQ3KCV3ugGJufr9hmH73XUBNhZRdKipZyaRLfPx8dqr
66,ERuxDEJ9RvfhtdjmtQMfyUQQgPbppKWTpRz9LetHCND2EZM,0.01000,2022-06-17T21:41:30.004Z,a3c4kQN1jXFWBPGRFFLJXyDFjBts7vhe7JiKgJgTZ1rXGDdmC
67,F4w3WyrsywpZp7b1YHPQmNQLvqoGy4oS6EK5fjzfSj5zzUG,0.01000,2022-05-17T08:57:00.106Z,a3chmVfmJFoXLFBu6UzBZi19j89KKNrPQGz81F1YfQ6t9CVuY


So only about 64 accounts moved KSM to Kintsugi (likely all vault owners)

Let's identify all the accounts who crowdloaned to Kintsugi!

In [8]:
crowdloans = """
query CrowloanersKintsugi {
  crowdloans(where: {para: {id_eq: "2092"}}) {
    contributions {
      amount
      timestamp
      account {
        id
        substrateAccount
      }
    }
  }
}
"""
r = requests.post("https://app.gc.subsquid.io/beta/parity-kusama/1/graphql", json={"query" : crowdloans}).json()
crd = pd.json_normalize(r['data']['crowdloans'][0]['contributions'])
crd['ksm'] = crd['amount'].apply(lambda x: float(x) / 1e12)
crowdloans = crd.groupby(['account.id', 'account.substrateAccount']).agg(
    dict(
        ksm=sum,
        timestamp=min
    )
)
crowdloans.describe()

,ksm
count,8013.000000
mean,24.959432
std,486.510228
min,0.100000
25%,0.111000
50%,1.000000
75%,2.100000
max,22688.000000


That is a much higher number of people: more than 8000 wallets. Mean amount of KSM contributed: 25. Let's see how many people covered the top 80% -- it's 1600, which confirms the famous 80/20 rule.

In [9]:
q = crowdloans.ksm.quantile(.8)
top_loaners = crowdloans[crowdloans.ksm>q]
top_loaners.reset_index(inplace=True)
top_loaners.rename(columns={"account.id":"kusama",
                            "timestamp": "first_seen"}, inplace=True)
top_loaners['kintsugi'] = top_loaners.kusama.apply(convert_ksm_to_kint)

top_loaners.loc[:, ['kusama', 'kintsugi', 'ksm', 'first_seen']].sort_values('ksm', ascending=False)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,kusama,kintsugi,ksm,first_seen
643,EwAhYwv7TJ1V8fxchsyZHNyturGYgoxhfEiiEnwCt7bWFjD,a3ca19hjMVGsXAWTU69n9rXAJg8Knea8ZYZ8QsabbwYGehfYx,22688.00000,2021-09-24T10:52:42.022Z
1439,HpofvM4WwTEfiAfD8tJPSoV4EQtUxHCbUK8eSHcweoudWS1,a3fTe858Vtm2kM5xAganUggaoqStQaqboSNCpon6HgJxxpnhW,18517.00000,2021-09-24T10:53:18.003Z
596,EkmdfH2Fc6XgPgDwMjye3Nsdj27CCSi9np8Kc7zYoCL2S3G,a3cPc5p4TdRg3MmTjQoe9wHACQwVdJ5mJzghpUwvfHTMPDXvg,17270.44313,2021-09-21T23:42:36.005Z
520,EVAMXLuZGhXcjPWb3JgX6a5qfPq2FH9usUZNuzW5XNQGvEw,a3c7zog8Lw6H3J7B24VCrpLMQcssM88bkkmNFYFoApBXTU9FC,15001.00000,2021-09-17T06:32:06.005Z
1295,HH5CgPv1RAgQMgpJh6p3NdT9juW5RkTgNTZyY14x3YEEY9W,a3euueqBMPEkC5jUKn8zzLcQmvxP2BK54XGMG8sojghhHS8R1,10000.00000,2021-09-16T11:15:06.007Z
...,...,...,...,...
214,DKHXv2J3o9DWZwuNC8dKkRn4ycpu6ikiUTRnYSc1rAXJxmT,a3ax7z4ojRcijBwjQqe2oczD6rC6Lzz3MZNM7wtFGkWKaWF7N,3.05000,2021-09-25T18:32:24.006Z
851,FfjPRZZNzgvQEJWJes4ojqNpUnP4CQhmvJAh8QCfLutsrL4,a3dJZqaLzkpGS5c61n6mF6ychbhFuA5jJcpBrrUCsQ14x58XX,3.05000,2021-09-25T10:54:48.008Z
1124,GhF5ETmUupvonU6Uzt6HgJntYMsEwbCiiVC14pQqTL4xBGU,a3eL5XPFCrjQSVAFbxSnGav67fkqPLpuoZcNtAQd5a7V89VgS,3.04570,2021-09-24T19:57:06.012Z
105,CuvFiWWHSSdnHEqnyejt1aMdefca1cNBe92rbB5GJ9TtKPu,a3aYkhsHwfG29Tf2MGRYvBFMgQs98ftvy2Y2j1vyjzxJX5ZWd,3.04000,2021-09-24T22:29:36.014Z


In [10]:
redeem_query = """query MyQuery {
  redeems(orderBy: request_timestamp_ASC) {
    id
    request {
      requestedAmountBacking
      timestamp
      height {
        absolute
        active
      }
    }
    userParachainAddress
    vault {
      accountId
      collateralToken
      wrappedToken
    }
    userBackingAddress
    bridgeFee
    btcTransferFee
    collateralPremium
    status
    execution {
      height {
        absolute
        active
      }
      timestamp
    }
    cancellation {
      timestamp
      slashedCollateral
      reimbursed
      height {
        absolute
        active
      }
    }
  }
}
"""


Let's use the official Kintsugi squid to download data about issue requests and redemptions. We have to ignore some other relevant ways that affect vaults: thefts and liquidations

In [11]:
issue_query = """
query MyQuery {
  issues(orderBy: request_timestamp_DESC, limit: 10000, offset: 0) {
    id
    request {
      amountWrapped
      bridgeFeeWrapped
      timestamp
      height {
        absolute
        active
      }
    }
    userParachainAddress
    vault {
      accountId
      collateralToken
      wrappedToken
    }
    vaultBackingAddress
    vaultWalletPubkey
    griefingCollateral
    status
    refund {
      amountPaid
      btcAddress
      btcFee
      executionHeight {
        absolute
        active
      }
      executionTimestamp
      id
      requestHeight {
        absolute
        active
      }
      requestTimestamp
    }
    execution {
      height {
        absolute
        active
      }
      amountWrapped
      bridgeFeeWrapped
      timestamp
    }
    cancellation {
      timestamp
      height {
        absolute
        active
      }
    }
  },
}
"""

In [12]:
# Obtain a list of all redemptions and summarize them a bit
r = requests.post(kintsugi, json={"query" : redeem_query}).json()
df_1 = pd.json_normalize(r['data']['redeems'])
df_1['btc'] = df_1['request.requestedAmountBacking'].apply(lambda x: float(x) / -1e8)
df_1['self'] = df_1.apply(is_equal, axis=1)
df_1['action'] = "redeem"

# Obtain a list of all issue executions and summarize them a bit
# issue_query = get_query_text_from_file("issue")
r = requests.post(kintsugi, json={"query" : issue_query}).json()
df_2 = pd.json_normalize(r['data']['issues'])
df_2['btc'] = df_2['request.amountWrapped'].apply(lambda x: float(x) / 1e8)
df_2['self'] = df_2.apply(is_equal, axis=1)
df_2['action'] = "issue"

cols_1 = set(df_1.columns) - set(df_2.columns)
cols_2 = set(df_2.columns) - set(df_1.columns)


The 80% quantile gives us the accounts responsible for 80% of the issues & redemptions

In [13]:
redeems = df_1.groupby('userParachainAddress').agg({'btc':sum}).sort_values('btc', ascending=True)
redeems['btc'] = redeems.btc.apply(lambda x: -x)
q = redeems.btc.quantile(.8)
top_redeemers = redeems[redeems.btc>q].index
add_label_to_list(top_redeemers, 'Top Redeemer')
redeems['btc'] = redeems.btc.apply(lambda x: -x)


In [14]:
issues = df_2.groupby('userParachainAddress').agg({'btc':sum}).sort_values('btc', ascending=False)
q = issues.btc.quantile(.8)
top_issues = issues[issues.btc>q].index
add_label_to_list(top_issues, 'Top Issuer')



In [15]:
issues[issues.btc>q].btc.sum()

53.67173071

In [16]:
self_issuers = df_2[df_2.self==True]


Let's consolidate this info and merge the two dataframes so we can net out the issue requests and the redeem requests.

In [17]:
merged_df = pd.concat([
    df_1.loc[df_1.status=='Completed', ['vault.accountId', 'btc', 'request.timestamp', 'self']],
    df_2.loc[df_2.status=='Completed', ['vault.accountId', 'btc', 'request.timestamp', 'self']],
                       ]).sort_values('request.timestamp')
# apportion BTC to self or other
merged_df['btc_self'] = merged_df.apply(lambda b: b.btc if b.self else 0, axis=1)
merged_df['btc_other'] = merged_df.apply(lambda b: b.btc if not b.self else 0, axis=1)
# add up the BTC depending on self/other
biggest_vaults = merged_df.groupby('vault.accountId').agg(dict(btc_self='sum', btc_other='sum', btc='sum')).sort_values('btc', ascending=False)
biggest_vaults['social'] = biggest_vaults.apply(lambda b: True if b.btc_self<0.2*b.btc else False, axis=1)
biggest_vaults['selfish'] = biggest_vaults.apply(lambda b: True if b.btc_self>0.8*b.btc else False, axis=1)
add_label_to_list(biggest_vaults.loc[biggest_vaults.social].index, 'Social')
add_label_to_list(biggest_vaults.loc[biggest_vaults.selfish].index, 'Selfish')


biggest_vaults.reset_index(inplace=True)
enrich_df(biggest_vaults, "vault.accountId")
total_btc = biggest_vaults.btc.sum()
biggest_vaults['share'] = biggest_vaults.btc.apply(lambda btc: f"{btc/total_btc:.1%}")


In [18]:
vaults = biggest_vaults.reset_index()
del vaults['index']
vaults = vaults.rename(columns={"vault.accountId":"vault"})
add_label_to_list(list(vaults.vault.iloc[0:20]), "Top 20 Vault")
add_label_to_list(list(vaults.vault.iloc[20:]), "Vault")
add_label_to_list(list(vaults.vault.loc[vaults.vault.isin(self_issuers.userParachainAddress)].iloc[0:20]), "Self Issuer")
enrich_df(vaults, 'vault')
vaults.iloc[0:25]

,vault,btc_self,btc_other,btc,social,selfish,label_vault.accountId,share,label_vault
0,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,0.898650,12.499693,13.398343,True,False,Top Issuer/Social,44.7%,Top Issuer/Social/Top 20 Vault/Self Issuer
1,a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3,4.161165,0.114068,4.275232,False,True,Lucky Friday/Top Redeemer/Top Issuer/Selfish,14.2%,Lucky Friday/Top Redeemer/Top Issuer/Selfish/T...
2,a3aGT3FRF1WgWtWdi8VmhB3YEJVE6XtrvT41TqcKFxZbuUvS1,-0.098253,1.433763,1.335510,True,False,@warinelly/Social,4.5%,@warinelly/Social/Top 20 Vault
3,a3dqzCRPDD9gwrXZ63FBa6hX2q6DsMcUNWPooK2mNNwTDqajq,0.744558,0.183030,0.927588,False,True,Top Redeemer/Top Issuer/Selfish,3.1%,Top Redeemer/Top Issuer/Selfish/Top 20 Vault/S...
4,a3bkRMWVb3C3JyXVFvfR5Brq435C6eqMr42YhA759JyKTzRzQ,-1.738186,2.610267,0.872081,True,False,Top Redeemer/Social,2.9%,Top Redeemer/Social/Top 20 Vault/Self Issuer
5,a3eKvTxY56smUwHU9vLpw9w5kSqpoPkJskU2tNxUSHAnntQTS,0.827986,-0.004999,0.822987,False,True,@mafux777/Top Issuer/Selfish,2.7%,@mafux777/Top Issuer/Selfish/Top 20 Vault/Self...
6,a3bihCTc88uby4Xm5D8JbTWszYGhvt34BDKrAamPVikRH8Qem,-0.021208,0.650076,0.628867,True,False,Social,2.1%,Social/Top 20 Vault
7,a3eFe9M2HbAgrQrShEDH2CEvXACtzLhSf4JGkwuT9SQ1EV4ti,0.000000,0.625398,0.625398,True,False,@paride/Social,2.1%,@paride/Social/Top 20 Vault
8,a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe,0.015880,0.583043,0.598924,True,False,pumpernickel/Top Redeemer/Social,2.0%,pumpernickel/Top Redeemer/Social/Top 20 Vault/...
9,a3fxCiEXfZJThTpHR5FPT3jngnFWQygAtw1ieDp2w9669v6eN,1.139777,-0.546023,0.593753,False,True,Lucky Friday/Top Redeemer/Top Issuer/Selfish,2.0%,Lucky Friday/Top Redeemer/Top Issuer/Selfish/T...


Now comes the time to analyse transfers from Kintsugi to Kintsugi, Karura and Moonriver.

In [19]:
kintsugix_query="""
query MyQuery {
  transfers(orderBy: timestamp_DESC) {
    amount
    from {
      karura
      kintsugi
      kusama
      moonriver
      id
    }
    fromChain
    timestamp
    to {
      id
      karura
      kintsugi
      kusama
      moonriver
    }
    toChain
    token
  }
}

"""

In [20]:
r = requests.post(kintsugi_x, json={"query" : kintsugix_query}).json()
xtoken_transfers = pd.json_normalize(r['data']['transfers'])


In [21]:
master_t = dict(
    KSM = 1e12,
    BTC = 1e8,
    KBTC = 1e8,
    KINT = 1e12,
)

def fix_currency(row):
    token = row['token']
    divisor = master_t.get(token, 1.0)
    row[token.lower()] = float(row.get('amount', 0.0)) / divisor
    return row



In [22]:
xtoken_transfers.rename(columns={"from.id":"from_id", "to.id":"to_id"}, inplace=True)
xtoken_transfers = xtoken_transfers.apply(fix_currency, axis=1)


There are a few accounts involved in crowdloans and other "shenanigans". Let's label them, too. 

In [23]:
shenanigans = xtoken_transfers.loc[xtoken_transfers.toChain==2092].groupby('from_id').agg(dict(kint=sum, to_id='nunique'))
shenanigans.sort_values('to_id', ascending=False).iloc[0:20]

,kint,to_id
from_id,,
a3aWepSsLfy67vUAAaKCiSnCj7G72feeTCgfKCBLivBWWTdFW,155593.222190,560
a3drECEeNJTEfPkMYCu1njch4tmMK1msjjMPbFsF7PSe5Qn5D,7634.632533,441
a3bkd4Z7FSnAVABG9JJV8UyESACpKgretzxMbYx7zLTLoVj2p,60653.233740,172
a3cvxyxiBBT88FkVZ7BYyhKPL9sNVdXYk6UBqk8GQnYzrK6xW,6745.682513,101
a3cUcEFSfpaK8spLYmzshHq1XcmyMtrJf66R1r84ngdQFiaXC,47003.688457,82
a3dtigkC2uf411XjMBN4KE9WSaNQUV3TxhkoRgYtj2VRHYHkT,82.154192,18
a3btcmyVE6ENtWVyHiX9QnorJfKfA2TsSCF43urDeNAWKueH6,568.994270,13
a3aPp8Ei4Qm3dTr57rRabVJhFKpGdzdxYkgFFgqVNZ7nqudyw,3.100000,12
a3dhHJLHVV2hxP7rgoeJsjNWDmnkvjTA1cVz6StP8YdZur8jm,48.000000,7


That analysis seems to suggest we should label the ones which sent more than 5 KINT as "shenanigans"...


In [24]:
shenanigans_list = shenanigans.loc[shenanigans.to_id>5].index
shenanigans.reset_index(inplace=True)
enrich_df(shenanigans, 'from_id')

add_label_to_list(shenanigans_list, "shenanigans")
enrich_df(shenanigans, 'from_id')


Who are the accounts that have sent KINT or KSM to the top 20 vaults? We shall exclude "shenanigans".

to_id: the top 20 vaults

from_id: the accounts sending KINT excluding shenanigans



In [25]:
from tensorflow.python.profiler import trace
funding_accounts = xtoken_transfers.loc[
                                        (xtoken_transfers.to_id.isin(vaults.iloc[0:20, 0])) & 
                                        ~(xtoken_transfers.from_id.isin(shenanigans_list)) & 
                                        (xtoken_transfers.toChain==2092)
                                        ].groupby(['to_id', 'from_id', 'from.kusama','fromChain']).agg(dict(kint=sum, ksm=sum, timestamp=min))
funding_accounts.reset_index(inplace=True)
funding_accounts.rename(columns=dict(to_id="vault", from_id="daddy"), inplace=True)


Let's label each daddy with his vault

In [26]:
for n, row in funding_accounts.iterrows():
  vault = row['vault'][-5:] # use only last 5 letters
  daddy = row['daddy']
  add_label_to_list([daddy], f'Daddy of {vault}')  
  add_label_to_list([row['vault']], f'{vault}')

In [27]:
add_label_to_list(funding_accounts.daddy, "Likely Vault Owner")
enrich_df(funding_accounts, 'vault')
enrich_df(funding_accounts, 'daddy')
funding_accounts.sort_values('ksm', ascending=False).loc[:, ['label_vault', 'label_daddy', 'kint', 'ksm']].iloc[0:20]


,label_vault,label_daddy,kint,ksm
12,@spazcoin/chaos DAO/Top Redeemer/Top Issuer/To...,Daddy of eCQvW/Likely Vault Owner,0.000000,174.710000
23,Simon Kraus/@sik/Top 20 Vault/Self Issuer/nPJjs,Simon Kraus/@sik/Top Redeemer/Selfish/Vault/Da...,210.000000,110.660000
3,Simon Kraus/@sik/Selfish/Top 20 Vault/Self Iss...,Simon Kraus/@sik/Top Redeemer/Selfish/Vault/Da...,14.900000,91.948160
10,Social/Top 20 Vault/Daddy of Poxg8/rEszH/Likel...,Social/Top 20 Vault/Poxg8/Daddy of rEszH/Likel...,6.803410,50.335594
11,@spazcoin/chaos DAO/Top Redeemer/Top Issuer/To...,Daddy of eCQvW/Likely Vault Owner,0.000000,30.000000
21,Simon Kraus/@sik/Top 20 Vault/Self Issuer/nPJjs,Simon Kraus/@sik/Selfish/Top 20 Vault/Self Iss...,855.000000,29.000000
2,Simon Kraus/@sik/Selfish/Top 20 Vault/Self Iss...,Top Redeemer/Top Issuer/Daddy of 74zdo/Daddy o...,0.000000,3.110000
22,Simon Kraus/@sik/Top 20 Vault/Self Issuer/nPJjs,Top Redeemer/Top Issuer/Daddy of 74zdo/Daddy o...,0.000000,3.110000
19,@mafux777/Top Issuer/Selfish/Top 20 Vault/Self...,Top Redeemer/Daddy of ntQTS/Likely Vault Owner,8.230000,0.940000
0,@warinelly/Social/Top 20 Vault/uUvS1,Daddy of uUvS1/Likely Vault Owner,0.000000,0.000000


Of the funders, which ones do we know from the KSM analysis?

In [28]:
k = kusama_transfers.loc[kusama_transfers.ksm>200]
add_label_to_list(k.kintsugi, 'K>200')
enrich_df(k, 'kintsugi')
k

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,kusama,ksm,first_seen,kintsugi,label_kintsugi
0,GDN8YBtPVRkjR7YtEKMUMXzBdtsNYWW7djdsxcQDqSZSxmM,10000.00000,2022-04-25T15:53:42.020Z,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,Top Issuer/Social/Top 20 Vault/Self Issuer/ceZ...
1,HExSfZUFJPkbUxpKQ5jM54TA7ThwfsCRBfgowmzzCcH8rV2,3446.75000,2022-05-02T15:20:06.029Z,a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3,Lucky Friday/Top Redeemer/Top Issuer/Selfish/T...
2,CrZxfsquEZeXLZUk7Uxdts8NpKz1oc7pdDXwHYCSeJ8SLgW,2960.09492,2022-04-26T20:29:06.018Z,a3aVQQpfHH49ACiLzDZP8w8eTA2oW7gvifX7E6dLsBJTBdzVR,Lucky Friday/Vault/Self Issuer/K>200
3,GcohCErDM7LjUfBkmNqu19CNzRUEp84DQP4nZeUhjrBHqNP,1067.96346,2022-04-25T17:29:30.016Z,a3eFe9M2HbAgrQrShEDH2CEvXACtzLhSf4JGkwuT9SQ1EV4ti,@paride/Social/Top 20 Vault/K>200
4,Ea9GXVeSWj1vxXutpMPtZAinuACMeWYyVfKHqKrn2gb11kj,993.73600,2022-04-26T07:15:42.025Z,a3cCyigH5pLJXcLKRNGFaBnx3a7diTXq9pPZ1TB8XWgqeCQvW,@spazcoin/chaos DAO/Top Redeemer/Top Issuer/To...
5,Cdcb6dodh71D8rCf3sWsokuXGkaS1YKcA7mgGWbDuSrHSFA,814.00000,2022-05-16T14:33:36.024Z,a3aGT3FRF1WgWtWdi8VmhB3YEJVE6XtrvT41TqcKFxZbuUvS1,@warinelly/Social/Top 20 Vault/uUvS1/K>200
6,EcdZSvpYwbhESMviUwYQb9WiZTgnTYrAsRLUf3GmXyzy5dH,811.65000,2022-04-25T10:08:12.007Z,a3cFU1biFvmBCup9SBvqihpvqVmwCtLsT1mK2dzqwWC94AjJw,Top Redeemer/Daddy of ntQTS/Likely Vault Owner...
7,HyWvJx6EUsRV7zwtt3m5sfok8Z56cnf2oymLgu9johaLdRh,760.83000,2022-05-23T06:35:30.013Z,a3fcMNTjXcJSwAVnTNKwwP7T8XM2bCW7FshsTW2hpUTrdXzed,@spazcoin/chaos DAO/Top Redeemer/Top Issuer/Se...
8,DMZC2aCdR5SjYTr9duiHFHWHSuqwfFUjHX1Qru9YLSe9H5e,666.48400,2022-05-01T20:50:54.012Z,a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe,pumpernickel/Top Redeemer/Social/Top 20 Vault/...
9,GCE5wFsJX3x3MpPrekqnXe2STTye3gdvxvjX5gSXL5YSZ74,607.50000,2022-05-20T19:26:54.016Z,a3dq4Y63JgLdTijbuL6f25mRMDfwVjw1EmrpRgRV7FzEbdxDB,Social/Top 20 Vault/K>200


In [29]:
add_label_to_list(top_loaners.kintsugi, 'Top Crowdloaner')
enrich_df(top_loaners, 'kintsugi')
top_loaners.sort_values('ksm', ascending=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,kusama,account.substrateAccount,ksm,first_seen,kintsugi,label_kintsugi
643,EwAhYwv7TJ1V8fxchsyZHNyturGYgoxhfEiiEnwCt7bWFjD,5ERZ3Ec3V6H5jUrWr14vfL1ykKa2k1znFHPyKaWyj5u6mCfs,22688.00000,2021-09-24T10:52:42.022Z,a3ca19hjMVGsXAWTU69n9rXAJg8Knea8ZYZ8QsabbwYGehfYx,Top Crowdloaner
1439,HpofvM4WwTEfiAfD8tJPSoV4EQtUxHCbUK8eSHcweoudWS1,5HKC1c1BtaSJv4MDSS5FVVSUue8egHU296UPFn1fTrbQtTYd,18517.00000,2021-09-24T10:53:18.003Z,a3fTe858Vtm2kM5xAganUggaoqStQaqboSNCpon6HgJxxpnhW,Top Crowdloaner
596,EkmdfH2Fc6XgPgDwMjye3Nsdj27CCSi9np8Kc7zYoCL2S3G,5EF9yLw9dF5bvjrnAevvk61sV8jsPXdXhQyNvwr34zyqHC4W,17270.44313,2021-09-21T23:42:36.005Z,a3cPc5p4TdRg3MmTjQoe9wHACQwVdJ5mJzghpUwvfHTMPDXvg,Top Crowdloaner
520,EVAMXLuZGhXcjPWb3JgX6a5qfPq2FH9usUZNuzW5XNQGvEw,5DyYhD12vugbs5a4pLVdd9D5h57bDaTyTVdozFiYbj9uXow7,15001.00000,2021-09-17T06:32:06.005Z,a3c7zog8Lw6H3J7B24VCrpLMQcssM88bkkmNFYFoApBXTU9FC,Daddy of ceZ9g/Likely Vault Owner/Top Crowdloaner
1295,HH5CgPv1RAgQMgpJh6p3NdT9juW5RkTgNTZyY14x3YEEY9W,5GmTYN43P49kehsNXzHm9RGT19dGGkwHDzcpasj7UFKjVW2J,10000.00000,2021-09-16T11:15:06.007Z,a3euueqBMPEkC5jUKn8zzLcQmvxP2BK54XGMG8sojghhHS8R1,Top Crowdloaner
...,...,...,...,...,...,...
214,DKHXv2J3o9DWZwuNC8dKkRn4ycpu6ikiUTRnYSc1rAXJxmT,5CofsbgRRS8Hkv8TbVKaRo4mvPLb6RuaG6cgPtAeY3x2ZuW9,3.05000,2021-09-25T18:32:24.006Z,a3ax7z4ojRcijBwjQqe2oczD6rC6Lzz3MZNM7wtFGkWKaWF7N,Top Crowdloaner
851,FfjPRZZNzgvQEJWJes4ojqNpUnP4CQhmvJAh8QCfLutsrL4,5FA7j7DgkdfzeaV4Xx41unUNftW9FXbXKYTRJU8FBYhQ8hPf,3.05000,2021-09-25T10:54:48.008Z,a3dJZqaLzkpGS5c61n6mF6ychbhFuA5jJcpBrrUCsQ14x58XX,Top Crowdloaner
1124,GhF5ETmUupvonU6Uzt6HgJntYMsEwbCiiVC14pQqTL4xBGU,5GBdQv7trYp148eeiJ53Piwnjx5dSGn2GLeScQYTMf7aCyok,3.04570,2021-09-24T19:57:06.012Z,a3eL5XPFCrjQSVAFbxSnGav67fkqPLpuoZcNtAQd5a7V89VgS,Top Crowdloaner
105,CuvFiWWHSSdnHEqnyejt1aMdefca1cNBe92rbB5GJ9TtKPu,5CQJbQAdf5Ri2dRQ2Gqgz4DMV4PNmLoBjGJHTvu7nVvy96sB,3.04000,2021-09-24T22:29:36.014Z,a3aYkhsHwfG29Tf2MGRYvBFMgQs98ftvy2Y2j1vyjzxJX5ZWd,Top Crowdloaner


What do people do with their KBTC? Let's convert the amounts into human readable form and sort by the biggest KBTC movers, including across chain!

In [30]:
my_currencies = ['kint', 'ksm', 'kbtc']

# def add_more_labels(my_currency):
agg = {}
for c in my_currencies:
  agg[c] = sum

top_transfer = xtoken_transfers.groupby(['from_id', 'fromChain','to_id', 'toChain']).agg(agg)
top_transfer.reset_index(inplace=True)


for my_currency in my_currencies:
  q = top_transfer[my_currency].quantile(.8)

  top_transfer_ids = top_transfer[top_transfer[my_currency]>q].from_id
  add_label_to_list(top_transfer_ids, f'Top {my_currency.upper()} Mover')

  top_transfer_ids = top_transfer[top_transfer[my_currency]>q].to_id
  add_label_to_list(top_transfer_ids, f'Top {my_currency.upper()} Sink')

enrich_df(top_transfer, 'from_id')
enrich_df(top_transfer, 'to_id')

top_transfer.sort_values('kbtc', ascending=False)


,from_id,fromChain,to_id,toChain,kint,ksm,kbtc,label_from_id,label_to_id
1634,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,2092,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,2000,0.00000,0.0,5.249603,@alibaba/Top Redeemer/Top Issuer/Daddy of dXze...,@alibaba/Top Redeemer/Top Issuer/Daddy of dXze...
1630,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,2092,0x63e33447838b9025e5cfdb35d4acc12a38342614,2023,0.00000,0.0,2.840376,@alibaba/Top Redeemer/Top Issuer/Daddy of dXze...,Top KBTC Sink
1617,a3dJZWg4s2ZXPVygL5xTTW6F93hoy6mFadKHKbQAkHnodw9uR,2092,a3bkRMWVb3C3JyXVFvfR5Brq435C6eqMr42YhA759JyKTzRzQ,2092,11.00000,0.0,1.735892,Top Issuer/Daddy of TzRzQ/Likely Vault Owner/T...,Top Redeemer/Social/Top 20 Vault/Self Issuer/D...
1668,a3dV3XyhVP3f1B1dwCvW7V8k87M2maqRxLpk8bb5f4TbPVSFD,2092,0x7da0419a1c47750f19440019cda780998b32268f,2023,3010.68100,0.0,1.649081,Top Issuer/Top KINT Mover/Top KINT Sink/Top KB...,Top KINT Sink/Top KBTC Sink
700,a3ar4dXg4JKaQ939WQCcv38xUFwU3PuW1FPuzsGAgC2GFYPxK,2092,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,2092,0.50000,0.0,1.586544,Top Issuer/Top KBTC Mover/Top KBTC Sink,@alibaba/Top Redeemer/Top Issuer/Daddy of dXze...
...,...,...,...,...,...,...,...,...,...
903,a3bfa2HQc6x1VusYg35mLcbJV65WGNUWTpmpX6mVqDNNLeSKN,2092,a3dybFygWZpf3pr1YQAxwAYXxNLwZjx14tB2FNVhFC9bXGyVa,2092,62.56000,0.0,0.000000,,
905,a3bftNoJVHrKNfozn8zVRLsXmhTmiV2Tsiqua3J3jypZnrXYS,2092,a3bkd4Z7FSnAVABG9JJV8UyESACpKgretzxMbYx7zLTLoVj2p,2092,163.98974,0.0,0.000000,Top KINT Mover/Top KINT Sink,shenanigans/Top KINT Mover/Top KINT Sink
906,a3bg8wNLomGQv9iDqrdsLspK2m4mYxY4jNRfQXExu18FghaGP,2092,a3dYzadG8C3ZxWt6vfemGtki53EPHWdrgJb24fbaQfsSj2LQJ,2092,0.14000,0.0,0.000000,,
907,a3bgHYH68DZqdJ22cJCgrCHzNVTbhZDCwYfdxo5FY7wtFAPq4,2092,a3dtigkC2uf411XjMBN4KE9WSaNQUV3TxhkoRgYtj2VRHYHkT,2092,0.00087,0.0,0.000000,,shenanigans/Top KINT Sink/Top KSM Mover/Top KS...


Let's try to see which accounts are likely CEX accounts, i.e. they receive large sums from many addresses. So we aggregate by number of distinct "from IDs" and sum up KINT. To understand "history", we also look at the first and last transfer for each account.

In [31]:
xtoken_transfers['first'] = xtoken_transfers.timestamp # duplicate to determine first transfer
cex = xtoken_transfers.groupby('to_id').agg({
    "from_id" : "nunique",
    "first" : min,
    "timestamp" : max,
    "kint" : sum,
    "kbtc" : sum,
}).sort_values('from_id', ascending=False)
cex

,from_id,first,timestamp,kint,kbtc
to_id,,,,,
a3aWepSsLfy67vUAAaKCiSnCj7G72feeTCgfKCBLivBWWTdFW,132,2022-02-21T13:02:42.294000Z,2022-06-03T00:44:30.542000Z,80505.557305,0.0
a3bkd4Z7FSnAVABG9JJV8UyESACpKgretzxMbYx7zLTLoVj2p,89,2022-02-20T14:13:12.300000Z,2022-06-18T18:38:42.529000Z,101483.558925,0.0
a3cUcEFSfpaK8spLYmzshHq1XcmyMtrJf66R1r84ngdQFiaXC,22,2022-06-05T03:50:24.392000Z,2022-06-18T10:21:00.464000Z,12115.607581,0.0
a3awFkWwydtvXZaQdQqpTeRAQpehjEe6wh57i6NTnka8xxKkw,20,2022-04-08T15:41:36.378000Z,2022-04-08T16:05:30.257000Z,139.997250,0.0
a3d28NjUAJzQdBSdv7W2CgNKD3wY7GPcYfcDyffCnGDAiz2Ef,19,2022-03-05T16:36:42.266000Z,2022-03-05T16:50:42.251000Z,91.180000,0.0
...,...,...,...,...,...
a3c3JbE5Bs59XeGhHnvYRMjLa83UXygGFFUVCdo6G2Dxvrckx,1,2022-03-25T05:39:30.370000Z,2022-03-25T05:39:30.370000Z,15.402142,0.0
a3c3HYX9mC8Cg44LazFKdbKgECxzD8VGaRYMKmwbJjxo7zzaw,1,2022-03-22T15:11:54.361000Z,2022-03-22T15:11:54.361000Z,2.255388,0.0
a3c3DoM4f3D5H4QgMKz1FgNomf8SPGQwR3JMdjoBpq7fQpvHF,1,2022-03-18T19:47:30.327000Z,2022-03-18T19:47:30.327000Z,1008.612870,0.0


In [32]:
cex.reset_index(inplace=True)
cex = cex.loc[(cex.kint>200)&( cex.from_id>10)]
add_label_to_list(cex.to_id, 'CEX')

With some additional criteria based on the shape of the data (from ID>10, KINT>200) 
It seems that the the top 4 accounts

* a3aWepSsLfy67vUAAaKCiSnCj7G72feeTCgfKCBLivBWWTdFW
* a3bkd4Z7FSnAVABG9JJV8UyESACpKgretzxMbYx7zLTLoVj2p
* a3cUcEFSfpaK8spLYmzshHq1XcmyMtrJf66R1r84ngdQFiaXC
* a3dtigkC2uf411XjMBN4KE9WSaNQUV3TxhkoRgYtj2VRHYHkT

 are CEX accounts, with TdFW replaced by FiaXC (look at timestamps). Maybe the exchange needed to change the wallet address for some reason. It seems plausible because there was a gap of about 48 hours. 

Now let's analyse the accounts by "from", counting how many "to" there are and summing up KINT, KBTC and KSM

In [33]:
agg['to_id']='nunique'
agg['timestamp']='min'
xtoken_transfers['last_seen'] = xtoken_transfers.timestamp # workaround to agg this col, too
agg['last_seen']='max'

top_transfer_from = xtoken_transfers.groupby(['from_id', 'fromChain','toChain']).agg(agg)
top_transfer_from.kint = top_transfer_from.kint.apply(lambda k: round(k))
top_transfer_from = top_transfer_from.reset_index(inplace=False)

exclude = top_transfer_from.sort_values('kint', ascending=False).iloc[0:2, 0]
list(exclude)
add_label_to_list(exclude, 'KINT System?')
enrich_df(top_transfer_from, 'from_id')

In [34]:
top_transfer_from.sort_values('kint', ascending=False)

,from_id,fromChain,toChain,kint,ksm,kbtc,to_id,timestamp,last_seen,label_from_id
461,a3cgeH7D28bBsHWJtUcHf7srz25o34gCKi8SZVjky6nMyEm83,2092,2092,1325000,0.0,0.0,2,2022-03-16T20:11:24.342000Z,2022-03-24T20:29:54.518000Z,Top KINT Mover/KINT System?
49,a3aWepSsLfy67vUAAaKCiSnCj7G72feeTCgfKCBLivBWWTdFW,2092,2092,155593,0.0,0.0,560,2022-02-21T13:48:30.573000Z,2022-06-18T23:24:12.656000Z,shenanigans/Top KINT Mover/Top KINT Sink/CEX/K...
300,a3bkd4Z7FSnAVABG9JJV8UyESACpKgretzxMbYx7zLTLoVj2p,2092,2092,60653,0.0,0.0,172,2022-02-20T00:37:12.309000Z,2022-06-18T23:15:54.317000Z,shenanigans/Top KINT Mover/Top KINT Sink/CEX
700,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,2092,2092,49655,0.0,0.0,3,2022-03-29T14:37:30.396000Z,2022-06-05T14:54:18.296000Z,Top Issuer/Social/Top 20 Vault/Self Issuer/ceZ...
420,a3cUcEFSfpaK8spLYmzshHq1XcmyMtrJf66R1r84ngdQFiaXC,2092,2092,47004,0.0,0.0,82,2022-02-17T17:37:06.708000Z,2022-06-18T11:29:06.480000Z,shenanigans/Top KINT Mover/Top KINT Sink/CEX
...,...,...,...,...,...,...,...,...,...,...
298,a3bk3zmgwXYVk15EkeWickKhuSCZus8ueU7F9xiHkBegGQYN3,2092,2092,0,0.0,0.0,1,2022-02-21T15:41:36.430000Z,2022-02-21T15:41:36.430000Z,
306,a3bmMn9BKUSWQ2hXstBpdY3RCMQvFaR7nifMHoWVymZn6jPQA,2092,2092,0,0.0,0.0,1,2022-03-26T08:35:00.510000Z,2022-03-26T08:35:00.510000Z,Top KINT Sink
797,a3eLrRNoqfcKtbx132x5qs7nikxiorh4yHhb7yjC62ufoAqsX,2092,2092,0,0.0,0.0,1,2022-03-30T08:36:54.271000Z,2022-03-30T08:36:54.271000Z,
796,a3eLVCvu74QYDneJdNQyER5fSo9GcrvohsyWtQRKM2hiHHLCS,2092,2092,0,0.0,0.0,1,2022-05-26T11:50:42.619000Z,2022-05-26T11:50:42.619000Z,


Let's bring in a temporal dimension and explore when accounts became active and when they last did something.

In [35]:
def calc_secs(row):
  return (pd.Timestamp(row['last_seen'])-pd.Timestamp(row['timestamp'])) // pd.Timedelta("1s")

In [36]:
enrich_df(top_transfer_from, 'from_id')
top_transfer_from['duration'] = top_transfer_from.apply(calc_secs, axis=1)
top_transfer_from.sort_values('duration', ascending=False)


,from_id,fromChain,toChain,kint,ksm,kbtc,to_id,timestamp,last_seen,label_from_id,duration
420,a3cUcEFSfpaK8spLYmzshHq1XcmyMtrJf66R1r84ngdQFiaXC,2092,2092,47004,0.0,0.000000,82,2022-02-17T17:37:06.708000Z,2022-06-18T11:29:06.480000Z,shenanigans/Top KINT Mover/Top KINT Sink/CEX,10432319
845,a3ec2HDHg824rqKYqXW3v5Np3z9XKhZkZZb9YdxRYJd9QVLqe,2092,2092,712,0.0,0.000000,1,2022-02-17T18:24:00.183000Z,2022-06-17T18:30:42.453000Z,Top Crowdloaner/Top KINT Mover,10368402
300,a3bkd4Z7FSnAVABG9JJV8UyESACpKgretzxMbYx7zLTLoVj2p,2092,2092,60653,0.0,0.000000,172,2022-02-20T00:37:12.309000Z,2022-06-18T23:15:54.317000Z,shenanigans/Top KINT Mover/Top KINT Sink/CEX,10276722
446,a3cdCSbJu5bjntmcpGDh67Jnvi54XmMm99X5QoXRZbnSinCyH,2092,2092,180,0.0,0.000000,1,2022-02-19T15:01:54.440000Z,2022-06-18T05:46:54.347000Z,Top Crowdloaner/Top KINT Mover,10248299
1048,a3fqG9iWWv15rxGRviVutvjLEy9Dk42Gm9mNoNzPVbUKtfKWP,2092,2092,904,0.0,0.000266,3,2022-02-20T21:13:30.212000Z,2022-06-18T13:48:00.387000Z,Top Crowdloaner/Top KINT Mover/Top KBTC Mover,10168470
...,...,...,...,...,...,...,...,...,...,...,...
449,a3cdPu3pFVPkwH1PYicAWKPKK4spLsecdGAXmDNzJjP7Z9fxm,2092,2092,9,0.0,0.000000,1,2022-04-07T07:37:12.212000Z,2022-04-07T07:37:12.212000Z,,0
450,a3cevVHdVrubHuuc17dzNah9EJSrT7zwLF5JTfdt1XBmzUsVr,2092,2092,4,0.0,0.000000,1,2022-04-27T00:50:30.500000Z,2022-04-27T00:50:30.500000Z,Top Crowdloaner,0
451,a3cewdxZHhq7ayJcuPUxABgUfUbr3Fga5gxNi9kdBrs6HjACF,2092,2092,1,0.0,0.000000,1,2022-03-18T10:19:36.348000Z,2022-03-18T10:19:36.348000Z,,0
452,a3ceycSRoqkqBmB5uEfRhhJhd89HVQcN2U3MPYzWbCdtEpAtb,2092,2092,33,0.0,0.000000,1,2022-04-14T12:59:42.867000Z,2022-04-14T12:59:42.867000Z,,0


Let's see how we can get on-chain data directly (without the squid) -- information about the available functions is available from the kint_substrate handler.

In [37]:
kint_substrate = SubstrateInterface(
    url="wss://api-kusama.interlay.io/parachain",
)
pd.DataFrame(kint_substrate.get_metadata_storage_functions())


,storage_name,storage_modifier,storage_default_scale,storage_default,documentation,module_id,module_prefix,module_name,spec_version,type_keys,type_hashers,type_value,type_class
0,Account,Default,"[65, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","{'nonce': 0, 'consumers': 0, 'providers': 0, '...",The full account information for a particular...,System,System,System,19,[scale_info::0],[Blake2_128Concat],scale_info::3,Map
1,ExtrinsicCount,Optional,"[4, 0]",None,Total extrinsics count for the current block.,System,System,System,19,[],[Twox64Concat],scale_info::4,Plain
2,BlockWeight,Default,"[96, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","{'normal': 0, 'operational': 0, 'mandatory': 0}",The current weight for the block.,System,System,System,19,[],[Twox64Concat],scale_info::7,Plain
3,AllExtrinsicsLen,Optional,"[4, 0]",None,Total length (in bytes) for all extrinsics pu...,System,System,System,19,[],[Twox64Concat],scale_info::4,Plain
4,BlockHash,Default,"[128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",0x00000000000000000000000000000000000000000000...,Map of block numbers to block hashes.,System,System,System,19,[scale_info::4],[Twox64Concat],scale_info::9,Map
...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,PageIndex,Default,"[64, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","{'begin_used': 0, 'end_used': 0, 'overweight_c...",The page index.,DmpQueue,DmpQueue,DmpQueue,19,[],[Twox64Concat],scale_info::479,Plain
216,Pages,Default,"[4, 0]",[],The queue pages.,DmpQueue,DmpQueue,DmpQueue,19,[scale_info::4],[Blake2_128Concat],scale_info::480,Map
217,Overweight,Optional,"[4, 0]",None,The overweight messages.,DmpQueue,DmpQueue,DmpQueue,19,[scale_info::8],[Blake2_128Concat],scale_info::481,Map
218,ConcreteFungibleBalances,Default,"[64, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0,Concrete fungible balances under a given loca...,UnknownTokens,UnknownTokens,UnknownTokens,19,"[scale_info::60, scale_info::60]","[Blake2_128Concat, Blake2_128Concat]",scale_info::6,Map


This is how I figure out how to use the VaultRegistry / Vaults storage method. I call "query_map" and inspect the first output. The first part of the output
is the ID (param) to call for the storage function.
The second part is the output.

In [38]:
kint_substrate = SubstrateInterface(url="wss://api-kusama.interlay.io/parachain")

acct = list(kint_substrate.query_map("VaultRegistry", "Vaults"))
acct[0]


[<scale_info::70(value={'account_id': 'a3d2HzjdJoePv7ttTkGJXS6X65k1aaG1Q9HXQrRU2ThLAZKZZ', 'currencies': {'collateral': {'Token': 'KSM'}, 'wrapped': {'Token': 'KBTC'}}})>,
 <scale_info::377(value={'id': {'account_id': 'a3d2HzjdJoePv7ttTkGJXS6X65k1aaG1Q9HXQrRU2ThLAZKZZ', 'currencies': {'collateral': {'Token': 'KSM'}, 'wrapped': {'Token': 'KBTC'}}}, 'wallet': {'addresses': [{'P2WPKHv0': '0x9a489168836ccdcc1e86a02779807daa927ff7a6'}]}, 'status': 'CommittedTheft', 'banned_until': None, 'to_be_issued_tokens': 0, 'issued_tokens': 0, 'to_be_redeemed_tokens': 0, 'to_be_replaced_tokens': 0, 'replace_collateral': 0, 'active_replace_collateral': 0, 'liquidated_collateral': 0})>]

In [39]:
kint_substrate = SubstrateInterface(
    url="wss://api-kusama.interlay.io/parachain",
)
p = {'account_id': 'a3eKvTxY56smUwHU9vLpw9w5kSqpoPkJskU2tNxUSHAnntQTS',
 'currencies': {'collateral': {'Token': 'KSM'}, 'wrapped': {'Token': 'KBTC'}}}
result = kint_substrate.query(
    module='VaultRegistry',
    storage_function='Vaults',
    params=[p]
)
result.serialize()

{'active_replace_collateral': 0,
 'banned_until': None,
 'id': {'account_id': 'a3eKvTxY56smUwHU9vLpw9w5kSqpoPkJskU2tNxUSHAnntQTS',
  'currencies': {'collateral': {'Token': 'KSM'},
   'wrapped': {'Token': 'KBTC'}}},
 'issued_tokens': 82128731,
 'liquidated_collateral': 0,
 'replace_collateral': 0,
 'status': {'Active': False},
 'to_be_issued_tokens': 0,
 'to_be_redeemed_tokens': 0,
 'to_be_replaced_tokens': 0,
 'wallet': {'addresses': [{'P2WPKHv0': '0x0825c01d4ccd3a8d8746064e886ed662ba70e391'},
   {'P2WPKHv0': '0x0c12c9299e991b9ff993015a8e40c329ff50a6dd'},
   {'P2WPKHv0': '0x0c1b1209e358a9ae4ca87863ecb218dec54614d2'},
   {'P2WPKHv0': '0x11f8e31b5644a052dd2f4dbedad5cdbb829818a3'},
   {'P2WPKHv0': '0x142929f7e233f1e1e4ac21d9312d5d58ef7d67c3'},
   {'P2WPKHv0': '0x1dade113727d63f4b168eb3fa3042aa6df5fa189'},
   {'P2WPKHv0': '0x1e3581a93453660adb5d24056eba778945142607'},
   {'P2WPKHv0': '0x1ea85f05614d6855161d036ae2bd0f5d865c6b70'},
   {'P2WPKHv0': '0x1fafdceb55c3c1882c8cdd48290fb214d43db131'

So now we can define a little function to get current-on-chain data like this.

In [40]:
def get_collateral_ksm(vault_id, kint_substrate):
  result = kint_substrate.query(
      module='Tokens',
      storage_function='Accounts',
      params=[vault_id,
              {'Token': 'KSM'}]
  )
  c = result.serialize().get('reserved') / 1e12
  return c

In [41]:
with SubstrateInterface(url="wss://kintsugi-rpc.dwellir.com") as kint_substrate:
  my_vaults = kint_substrate.query_map("VaultRegistry", "Vaults")
my_vaults = [r[1].serialize() for r in my_vaults]


In [42]:
v = pd.json_normalize(my_vaults)
v.rename(columns={"id.account_id":"vault", "status.Active":"active"}, inplace=True)

master_t = dict(
    KSM = 1e12,
    BTC = 1e8,
    KBTC = 1e8,
    KINT = 1e12,
)

v['issued_kbtc'] = v.issued_tokens.apply(lambda x:round( x/1e8, 6))
v['liquidated_ksm'] = v.liquidated_collateral.apply(lambda x: round(x/1e12))
 
master_2 = v.sort_values('issued_kbtc', ascending=False).loc[v.issued_kbtc>.1, ['vault', 'status', 'active', 'issued_kbtc', 'liquidated_ksm']]
kint_substrate = SubstrateInterface(url="wss://kintsugi-rpc.dwellir.com")

master_2['collateral_ksm'] = master_2.vault.apply(get_collateral_ksm, kint_substrate=kint_substrate)
master_2['collateral_ksm'] = master_2['collateral_ksm'].apply(round)

So this list of vaults is up-to-date and accurate.

In [43]:
enrich_df(master_2, 'vault')
master_2

,vault,status,active,issued_kbtc,liquidated_ksm,collateral_ksm,label_vault
21,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,NaN,False,10.450110,0,10994,Top Issuer/Social/Top 20 Vault/Self Issuer/ceZ...
13,a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3,NaN,False,4.278527,0,4167,Lucky Friday/Top Redeemer/Top Issuer/Selfish/T...
47,a3eKvTxY56smUwHU9vLpw9w5kSqpoPkJskU2tNxUSHAnntQTS,NaN,False,0.821287,0,837,@mafux777/Top Issuer/Selfish/Top 20 Vault/Self...
37,a3aGT3FRF1WgWtWdi8VmhB3YEJVE6XtrvT41TqcKFxZbuUvS1,NaN,True,0.821042,0,798,@warinelly/Social/Top 20 Vault/uUvS1/K>200/Top...
4,a3bkRMWVb3C3JyXVFvfR5Brq435C6eqMr42YhA759JyKTzRzQ,NaN,False,0.667683,0,1275,Top Redeemer/Social/Top 20 Vault/Self Issuer/D...
58,a3eFe9M2HbAgrQrShEDH2CEvXACtzLhSf4JGkwuT9SQ1EV4ti,NaN,False,0.630625,0,758,@paride/Social/Top 20 Vault/K>200/Top KINT Mov...
26,a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe,NaN,False,0.599667,0,604,pumpernickel/Top Redeemer/Social/Top 20 Vault/...
19,a3fxCiEXfZJThTpHR5FPT3jngnFWQygAtw1ieDp2w9669v6eN,NaN,False,0.594895,0,579,Lucky Friday/Top Redeemer/Top Issuer/Selfish/T...
14,a3dq4Y63JgLdTijbuL6f25mRMDfwVjw1EmrpRgRV7FzEbdxDB,NaN,True,0.564291,0,577,Social/Top 20 Vault/K>200/Top KINT Mover
39,a3bihCTc88uby4Xm5D8JbTWszYGhvt34BDKrAamPVikRH8Qem,NaN,True,0.534115,0,517,Social/Top 20 Vault/H8Qem/Top KINT Mover/Top K...


Now we can get the price of BTC in KSM to determine collateralization

In [44]:
kint_substrate = SubstrateInterface(url="wss://api-kusama.interlay.io/parachain")

oracle = list(kint_substrate.query_map("Oracle", "Aggregate"))
ksm_per_btc = oracle[1][1].serialize()/1e22
ksm_per_btc

395.65100416224857

In [45]:
def get_account_balances(id, token):
  result = kint_substrate.query(
      module='Tokens',
      storage_function='Accounts',
      params=[id,
              {'Token': token}]
  )
  c = result.serialize().get('free') / 1e8
  return c

master_2['kbtc_balance'] = master_2.vault.apply(get_account_balances, token='KBTC')


In [46]:
master_2

,vault,status,active,issued_kbtc,liquidated_ksm,collateral_ksm,label_vault,kbtc_balance
21,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,NaN,False,10.450110,0,10994,Top Issuer/Social/Top 20 Vault/Self Issuer/ceZ...,0.036140
13,a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3,NaN,False,4.278527,0,4167,Lucky Friday/Top Redeemer/Top Issuer/Selfish/T...,4.175044
47,a3eKvTxY56smUwHU9vLpw9w5kSqpoPkJskU2tNxUSHAnntQTS,NaN,False,0.821287,0,837,@mafux777/Top Issuer/Selfish/Top 20 Vault/Self...,0.117700
37,a3aGT3FRF1WgWtWdi8VmhB3YEJVE6XtrvT41TqcKFxZbuUvS1,NaN,True,0.821042,0,798,@warinelly/Social/Top 20 Vault/uUvS1/K>200/Top...,0.000700
4,a3bkRMWVb3C3JyXVFvfR5Brq435C6eqMr42YhA759JyKTzRzQ,NaN,False,0.667683,0,1275,Top Redeemer/Social/Top 20 Vault/Self Issuer/D...,0.006389
58,a3eFe9M2HbAgrQrShEDH2CEvXACtzLhSf4JGkwuT9SQ1EV4ti,NaN,False,0.630625,0,758,@paride/Social/Top 20 Vault/K>200/Top KINT Mov...,0.001328
26,a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe,NaN,False,0.599667,0,604,pumpernickel/Top Redeemer/Social/Top 20 Vault/...,0.086383
19,a3fxCiEXfZJThTpHR5FPT3jngnFWQygAtw1ieDp2w9669v6eN,NaN,False,0.594895,0,579,Lucky Friday/Top Redeemer/Top Issuer/Selfish/T...,1.142878
14,a3dq4Y63JgLdTijbuL6f25mRMDfwVjw1EmrpRgRV7FzEbdxDB,NaN,True,0.564291,0,577,Social/Top 20 Vault/K>200/Top KINT Mover,0.000536
39,a3bihCTc88uby4Xm5D8JbTWszYGhvt34BDKrAamPVikRH8Qem,NaN,True,0.534115,0,517,Social/Top 20 Vault/H8Qem/Top KINT Mover/Top K...,0.000164


Let's see if the daddies have a lot of KBTC sitting around...
This helps us understand if a Daddy is really active as a vault owner. Most likely, only the ones with some KBTC are actually using a vault. The other "daddies" include, for example, a CEX account used to send some KINT to a vault.
So the "daddy" label should not be relied on without further analysis.

In [47]:
f = funding_accounts.groupby(['daddy','label_daddy']).size().reset_index()
f['kbtc_balance'] = f.daddy.apply(get_account_balances, token='KBTC')
f['kint_balance'] = f.daddy.apply(get_account_balances, token='KINT')
f['kint_balance'] = f['kint_balance'].apply(lambda x: round(x/1e4))
f.sort_values('kbtc_balance', ascending=False)


,daddy,label_daddy,0,kbtc_balance,kint_balance
18,a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3,Lucky Friday/Top Redeemer/Top Issuer/Selfish/T...,2,4.175044,5
9,a3bzJWesF683HSZkvpyXWcejKWVjroyPyF6xGL1GEJBQy5Dh5,Top Redeemer/Top Issuer/Social/Vault/Daddy of ...,1,0.223724,26
2,a3adPv7McJ11sBVApm9ZcR52wpQAKBLFjd2n9E6DH1cBkccMg,Top Issuer/Selfish/Top 20 Vault/Self Issuer/Da...,1,0.180143,5
12,a3cFU1biFvmBCup9SBvqihpvqVmwCtLsT1mK2dzqwWC94AjJw,Top Redeemer/Daddy of ntQTS/Likely Vault Owner,1,0.129737,15
7,a3bkRMWVb3C3JyXVFvfR5Brq435C6eqMr42YhA759JyKTzRzQ,Top Redeemer/Social/Top 20 Vault/Self Issuer/D...,2,0.006389,0
3,a3addPTx9ngWGKq3dguw7vs7NA2PimcDUHWJ32HsuoFL74zdo,Simon Kraus/@sik/Selfish/Top 20 Vault/Self Iss...,1,0.002036,11
20,a3fxS24Bduq8cxT4oV7RQ6q6vfEdSEkaapER75BS4rF7U6Q63,Simon Kraus/@sik/Top Redeemer/Selfish/Vault/Da...,2,0.001732,15
13,a3cgeH7D28bBsH77KFYdoMgyiXUHdk98XT5M2Wv5EgC45Kqya,Daddy of ntQTS/Likely Vault Owner,1,0.001000,14208
10,a3c7uUVvpdHUySP3oJ3W16uQPN9cyAa7AxLmMmGsQAQmrEszH,Social/Top 20 Vault/Daddy of Poxg8/rEszH/Likel...,1,0.000587,1611
8,a3bmFphgwxL9SRc9gNTmWmR2eYGsfN8MdsRur1r6hbRQYnECq,Daddy of ceZ9g/Likely Vault Owner,1,0.000000,5


Let's now use the Oracle price to figure out the best and the worst vaults in terms of ratio...

In [48]:
master_2['coll_ratio'] = 100 * (master_2['collateral_ksm'] / ksm_per_btc ) / master_2['issued_kbtc']
master_2['coll_ratio'] = master_2['coll_ratio'].apply(round)
master_2.sort_values('coll_ratio', ascending=False).reset_index().loc[:,set(master_2.columns)-{"index", "status", "liquidated_ksm"}]

,coll_ratio,label_vault,active,collateral_ksm,kbtc_balance,issued_kbtc,vault
0,483,Top Redeemer/Social/Top 20 Vault/Self Issuer/D...,False,1275,0.006389,0.667683,a3bkRMWVb3C3JyXVFvfR5Brq435C6eqMr42YhA759JyKTzRzQ
1,370,@spazcoin/chaos DAO/Top Redeemer/Top Issuer/To...,False,729,0.105535,0.497328,a3cCyigH5pLJXcLKRNGFaBnx3a7diTXq9pPZ1TB8XWgqeCQvW
2,338,Social/Vault/Self Issuer/Top KINT Mover/Top KS...,False,270,0.000442,0.201952,a3fRJZLKJEsNY2fcDDLtzh7MWAE5DbVykyEfBbKdPbJSGLQjh
3,304,@paride/Social/Top 20 Vault/K>200/Top KINT Mov...,False,758,0.001328,0.630625,a3eFe9M2HbAgrQrShEDH2CEvXACtzLhSf4JGkwuT9SQ1EV4ti
4,301,Selfish/Vault/Self Issuer/Top KINT Mover,False,138,0.095098,0.115836,a3cS7bP56bj11Yrfxt3TZFGjo96R7eJH6WUNYBxg1dx55jCJm
5,266,Top Issuer/Social/Top 20 Vault/Self Issuer/ceZ...,False,10994,0.036140,10.450110,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g
6,261,@timbotronic/Top Issuer/Selfish/Vault/Self Iss...,False,226,0.000014,0.218633,a3aPvmjypKaDtjRgYbDL2CCkseYR3SLwvPevL6j7wF67aFtV4
7,258,@mafux777/Top Issuer/Selfish/Top 20 Vault/Self...,False,837,0.117700,0.821287,a3eKvTxY56smUwHU9vLpw9w5kSqpoPkJskU2tNxUSHAnntQTS
8,258,Social/Top 20 Vault/K>200/Top KINT Mover,True,577,0.000536,0.564291,a3dq4Y63JgLdTijbuL6f25mRMDfwVjw1EmrpRgRV7FzEbdxDB
9,258,@spazcoin/chaos DAO/Top Redeemer/Top Issuer/Se...,False,435,0.310009,0.426851,a3fcMNTjXcJSwAVnTNKwwP7T8XM2bCW7FshsTW2hpUTrdXzed


In [49]:
master_2.kbtc_balance.sum()

6.93264634

In [50]:
f.loc[~f.daddy.isin(master_2.vault), :].kbtc_balance.sum()

0.35619371

# Next few cells are for ad-hoc analysis

Most of the code above was used in a visualisation using Python / Flask / Dash. 

You can clone this workbook and use it for your own analysis, if you like.

In [51]:
from datetime import datetime, timedelta, timezone

now = datetime.now(timezone.utc)
earlier = pd.to_datetime(now - timedelta(days=7))
earlier

Timestamp('2022-06-26 15:27:13.260745+0000', tz='UTC')

In [52]:
v="a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe"
xtoken_transfers['timestamp_dt'] = xtoken_transfers.timestamp.apply(pd.to_datetime)
xtoken_transfers['ago'] = xtoken_transfers.timestamp_dt.apply(lambda t: now-t)
enrich_df(xtoken_transfers, 'to_id')
xtoken_transfers.loc[(xtoken_transfers['from_id']==v) & 
                     (xtoken_transfers.token=='KINT') &
                     (xtoken_transfers.timestamp_dt > earlier), ['ago','kint', 'kbtc', 'timestamp', 'to_id', 'toChain']]


,ago,kint,kbtc,timestamp,to_id,toChain


In [53]:
_.kint.sum()

0.0

In [54]:
kts['a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3']

['Lucky Friday',
 'Top Redeemer',
 'Top Issuer',
 'Selfish',
 'Top 20 Vault',
 'Self Issuer',
 'Daddy of g5rez',
 'LLDD3',
 'Daddy of 9v6eN',
 'Likely Vault Owner',
 'K>200',
 'Top KINT Mover']

In [55]:
premiums = """
query MyQuery {
  redeemExecutions(orderBy: timestamp_DESC, where: {redeem: {collateralPremium_gt: "0"}}) {
    timestamp
    redeem {
      status
      userBackingAddress
      userParachainAddress
      vault {
        accountId
      }
      collateralPremium
      request {
        requestedAmountBacking
      }
    }
  }
}

"""

In [56]:
r = requests.post(kintsugi, json={"query" : premiums}).json()
premium_redeems = pd.json_normalize(r['data']['redeemExecutions'])
premium_redeems['ksm_premium'] = premium_redeems['redeem.collateralPremium'].apply(lambda x: float(x)/1e12)
premium_redeems['btc_requested'] = premium_redeems['redeem.request.requestedAmountBacking'].apply(lambda x: float(x)/1e8)

premium_redeems.loc[premium_redeems.timestamp>"2022-06-10"].groupby('redeem.userParachainAddress').agg(
    {
        "ksm_premium": sum,
        "btc_requested": sum,
        "timestamp": max,
     
    }
).reset_index().rename(columns={"redeem.userParachainAddress":"user"}, inplace=False).sort_values('ksm_premium', ascending=False)


,user,ksm_premium,btc_requested,timestamp
5,a3cFU1biFvmBCup9SBvqihpvqVmwCtLsT1mK2dzqwWC94AjJw,36.050921,1.439581,2022-06-12T04:37:12.502Z
6,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,7.449030,0.298392,2022-06-12T04:40:06.463Z
4,a3bzJWesF683HSZkvpyXWcejKWVjroyPyF6xGL1GEJBQy5Dh5,5.452749,0.217518,2022-06-12T04:37:12.502Z
0,a3ar4dXg4JKaQ939WQCcv38xUFwU3PuW1FPuzsGAgC2GFYPxK,4.976767,0.198908,2022-06-12T04:37:12.502Z
1,a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe,3.733509,0.149104,2022-06-12T04:37:12.502Z
7,a3fVzyUt9RGMLbf3dD5C11GybmJV8mHxKuyKqodhdBbmZuUrY,1.769771,0.065336,2022-06-14T05:03:18.358Z
2,a3btcmyVE6ENtWVyHiX9QnorJfKfA2TsSCF43urDeNAWKueH6,0.275274,0.010849,2022-06-13T02:32:06.252Z
3,a3bzFrZ5kXYpaaD5NbapUDSfjZPQTWFKGwSbMmGeRAL8BGrCs,0.219618,0.007923,2022-06-14T05:03:18.358Z


In [57]:
redeem_cxl = """
query MyQuery {
  redeemCancellations(orderBy: timestamp_DESC, where: {redeem: {collateralPremium_gt: "75639044748"}}) {
    timestamp
    redeem {
      status
      userBackingAddress
      userParachainAddress
      vault {
        accountId
      }
      collateralPremium
      request {
        requestedAmountBacking
      }
    }
  }
}
"""

In [58]:
r = requests.post(kintsugi, json={"query" : premiums}).json()
premium_redeems_x = pd.json_normalize(r['data']['redeemExecutions'])
premium_redeems_x['ksm_premium'] = premium_redeems_x['redeem.collateralPremium'].apply(lambda x: float(x)/1e12)
premium_redeems_x['btc_requested'] = premium_redeems_x['redeem.request.requestedAmountBacking'].apply(lambda x: float(x)/1e8)
premium_redeems_x.groupby('redeem.userParachainAddress').agg(
    {
        "ksm_premium": sum,
        "btc_requested": sum,
        "timestamp": max,
     
    }
).reset_index().rename(columns={"redeem.userParachainAddress":"user"}, inplace=False).sort_values('ksm_premium', ascending=False)


,user,ksm_premium,btc_requested,timestamp
16,a3dqzCRPDD9gwrXZ63FBa6hX2q6DsMcUNWPooK2mNNwTDqajq,45.080122,2.273760,2022-05-14T20:19:30.305Z
11,a3cFU1biFvmBCup9SBvqihpvqVmwCtLsT1mK2dzqwWC94AjJw,40.570900,1.687699,2022-06-12T04:37:12.502Z
14,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,14.024796,0.617199,2022-06-12T04:40:06.463Z
9,a3bzJWesF683HSZkvpyXWcejKWVjroyPyF6xGL1GEJBQy5Dh5,11.846199,0.565476,2022-06-12T04:37:12.502Z
18,a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3,10.225996,0.524853,2022-05-15T10:13:18.321Z
19,a3fTkM2XKvxD4rGA4UHt8Efr9MgvLZH6mvYGtKdMctaLr4fdV,9.340200,0.634175,2022-05-03T00:10:24.620Z
4,a3ar4dXg4JKaQ939WQCcv38xUFwU3PuW1FPuzsGAgC2GFYPxK,4.976767,0.198908,2022-06-12T04:37:12.502Z
5,a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe,4.055474,0.165112,2022-06-12T04:37:12.502Z
20,a3fVzyUt9RGMLbf3dD5C11GybmJV8mHxKuyKqodhdBbmZuUrY,2.082392,0.079109,2022-06-14T05:03:18.358Z
21,a3fxCiEXfZJThTpHR5FPT3jngnFWQygAtw1ieDp2w9669v6eN,1.337879,0.068667,2022-05-15T10:13:18.321Z


In [59]:
df = _
enrich_df(df, 'user')
df.ksm_premium.sum() / df.btc_requested.sum() / 400

0.05229388166532546

In [60]:
df

,user,ksm_premium,btc_requested,timestamp,label_user
16,a3dqzCRPDD9gwrXZ63FBa6hX2q6DsMcUNWPooK2mNNwTDqajq,45.080122,2.273760,2022-05-14T20:19:30.305Z,Top Redeemer/Top Issuer/Selfish/Top 20 Vault/S...
11,a3cFU1biFvmBCup9SBvqihpvqVmwCtLsT1mK2dzqwWC94AjJw,40.570900,1.687699,2022-06-12T04:37:12.502Z,Top Redeemer/Daddy of ntQTS/Likely Vault Owner...
14,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,14.024796,0.617199,2022-06-12T04:40:06.463Z,@alibaba/Top Redeemer/Top Issuer/Daddy of dXze...
9,a3bzJWesF683HSZkvpyXWcejKWVjroyPyF6xGL1GEJBQy5Dh5,11.846199,0.565476,2022-06-12T04:37:12.502Z,Top Redeemer/Top Issuer/Social/Vault/Daddy of ...
18,a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3,10.225996,0.524853,2022-05-15T10:13:18.321Z,Lucky Friday/Top Redeemer/Top Issuer/Selfish/T...
19,a3fTkM2XKvxD4rGA4UHt8Efr9MgvLZH6mvYGtKdMctaLr4fdV,9.340200,0.634175,2022-05-03T00:10:24.620Z,Top Redeemer/Top Issuer/Daddy of 74zdo/Daddy o...
4,a3ar4dXg4JKaQ939WQCcv38xUFwU3PuW1FPuzsGAgC2GFYPxK,4.976767,0.198908,2022-06-12T04:37:12.502Z,Top Issuer/Top KBTC Mover/Top KBTC Sink
5,a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe,4.055474,0.165112,2022-06-12T04:37:12.502Z,pumpernickel/Top Redeemer/Social/Top 20 Vault/...
20,a3fVzyUt9RGMLbf3dD5C11GybmJV8mHxKuyKqodhdBbmZuUrY,2.082392,0.079109,2022-06-14T05:03:18.358Z,Top KINT Mover/Top KINT Sink/Top KSM Mover/Top...
21,a3fxCiEXfZJThTpHR5FPT3jngnFWQygAtw1ieDp2w9669v6eN,1.337879,0.068667,2022-05-15T10:13:18.321Z,Lucky Friday/Top Redeemer/Top Issuer/Selfish/T...


In [61]:
# retries only
retries = """
query MyQuery {
  redeems(where: {cancellation: {slashedCollateral_gt: "0"}}, orderBy: collateralPremium_DESC) {
    status
    vault {
      accountId
      registrationTimestamp
    }
    collateralPremium
    request {
      requestedAmountBacking
      timestamp
    }
    userParachainAddress
    cancellation {
      reimbursed
      timestamp
      slashedCollateral
    }
  }
}

"""

In [62]:
r = requests.post(kintsugi, json={"query" : retries}).json()
premium_redeems_x = pd.json_normalize(r['data']['redeems'])
premium_redeems_x['ksm_premium'] = premium_redeems_x['collateralPremium'].apply(lambda x: float(x)/1e12)
premium_redeems_x['ksm_slashed'] = premium_redeems_x['cancellation.slashedCollateral'].apply(lambda x: float(x)/1e12)
premium_redeems_x['btc_requested'] = premium_redeems_x['request.requestedAmountBacking'].apply(lambda x: float(x)/1e8)
premium_redeems_x.groupby(['vault.accountId','userParachainAddress', 'status', ]).agg(
    {
        "ksm_premium": sum,
        "ksm_slashed": sum,
        "btc_requested": sum,
        "cancellation.timestamp": max,
     
    }
).reset_index().rename(columns={"userParachainAddress":"user",
                                "cancellation.timestamp": "last_action"
                            }, inplace=False).sort_values('ksm_slashed', ascending=False)


,vault.accountId,user,status,ksm_premium,ksm_slashed,btc_requested,last_action
4,a3b3EwCtmURY7K3d6aoWzouHriGfTsvCP2axuMVGpRpkPoxg8,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,Reimbursed,0.000000,20.083839,0.063478,2022-05-06T06:19:54.316Z
9,a3dqzCRPDD9gwrXZ63FBa6hX2q6DsMcUNWPooK2mNNwTDqajq,a3cFU1biFvmBCup9SBvqihpvqVmwCtLsT1mK2dzqwWC94AjJw,Retried,9.556306,19.525180,0.490481,2022-05-14T05:45:42.273Z
7,a3chQwNk9sxP4WHnm82cD6a5AR2tXJ7bmJ88yZGQp4Ay957PB,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,Retried,3.737476,7.915483,0.198931,2022-05-14T05:23:42.583Z
8,a3dqzCRPDD9gwrXZ63FBa6hX2q6DsMcUNWPooK2mNNwTDqajq,a3bAsW2TQb1c7HrcZd2y81SdCVSGEA5pRkfg576G2EyxQupbw,Reimbursed,0.308839,7.005320,0.015851,2022-05-14T06:25:48.346Z
5,a3bBUocXg2cDMctxmEUaQPteH67NyLqxrYwX7soHuZaCHV5nE,a3bzJWesF683HSZkvpyXWcejKWVjroyPyF6xGL1GEJBQy5Dh5,Retried,0.377534,2.594198,0.063188,2022-06-25T20:52:12.295Z
0,a3aGT3FRF1WgWtWdi8VmhB3YEJVE6XtrvT41TqcKFxZbuUvS1,a3aPvmjypKaDtjRgYbDL2CCkseYR3SLwvPevL6j7wF67aFtV4,Reimbursed,0.000000,2.280142,0.004902,2022-05-14T12:51:18.485Z
10,a3e2AF9D4WUhuZPZvNiAkW2eWGLfMWQfBYWzwgk8bhmHF3LDc,a3e22ukKfJ4ZPNzXbpfrcndUgPHbxw4BUox4t4J7BeK6tJe7q,Reimbursed,0.000000,1.922981,0.004804,2022-05-25T11:09:06.352Z
6,a3bBUocXg2cDMctxmEUaQPteH67NyLqxrYwX7soHuZaCHV5nE,a3crPwJ717zKVvpP8MWscjfVoXgwETmLhpAppat6P5fPLqKSb,Reimbursed,0.000000,1.374952,0.004929,2022-04-06T15:32:24.269Z
1,a3aGT3FRF1WgWtWdi8VmhB3YEJVE6XtrvT41TqcKFxZbuUvS1,a3cFU1biFvmBCup9SBvqihpvqVmwCtLsT1mK2dzqwWC94AjJw,Retried,0.000000,0.902337,0.029994,2022-05-08T08:52:48.305Z
2,a3aGT3FRF1WgWtWdi8VmhB3YEJVE6XtrvT41TqcKFxZbuUvS1,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,Retried,0.000000,0.804636,0.019833,2022-05-14T09:36:00.368Z


In [63]:
df = _
enrich_df(df, 'user')
df.ksm_slashed.sum() / df.btc_requested.sum() / 400

0.1795153097543407

In [64]:
q = """
query MyQuery {
  redeems(orderBy: request_timestamp_DESC, where: {collateralPremium_gt: "0"}) {
    request {
      requestedAmountBacking
      timestamp
    }
    status
    userParachainAddress
    vault {
      accountId
    }
    collateralPremium
  }
}
"""



In [65]:
r = requests.post(kintsugi, json={"query" : q}).json()
premium_redeems_x = pd.json_normalize(r['data']['redeems'])
premium_redeems_x['ksm_premium'] = premium_redeems_x['collateralPremium'].apply(lambda x: float(x)/1e12)
premium_redeems_x['btc_requested'] = premium_redeems_x['request.requestedAmountBacking'].apply(lambda x: float(x)/1e8)
premium_redeems_x.groupby(['userParachainAddress', 'status', 'vault.accountId']).agg(
    {
        "ksm_premium": sum,
        "btc_requested": sum,
     "request.timestamp": max,
     
    }
).reset_index().rename(columns={"userParachainAddress":"user",
                                "request.timestamp": "last_action"
                            }, inplace=False).sort_values('last_action', ascending=False)


,user,status,vault.accountId,ksm_premium,btc_requested,last_action
13,a3bzFrZ5kXYpaaD5NbapUDSfjZPQTWFKGwSbMmGeRAL8BGrCs,Completed,a3bzFrZ5kXYpaaD5NbapUDSfjZPQTWFKGwSbMmGeRAL8BGrCs,0.813513,0.038405,2022-06-13T08:31:24.536Z
42,a3fVzyUt9RGMLbf3dD5C11GybmJV8mHxKuyKqodhdBbmZuUrY,Completed,a3bzFrZ5kXYpaaD5NbapUDSfjZPQTWFKGwSbMmGeRAL8BGrCs,1.479197,0.053725,2022-06-13T07:23:24.258Z
10,a3btcmyVE6ENtWVyHiX9QnorJfKfA2TsSCF43urDeNAWKueH6,Completed,a3b5jAhU7CKnBKA3y92HkiCqRTbSKRMK5kN3RVPXcKoH4SDaC,0.132526,0.004944,2022-06-13T01:09:54.238Z
5,a3ar4dXg4JKaQ939WQCcv38xUFwU3PuW1FPuzsGAgC2GFYPxK,Completed,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,4.976767,0.198908,2022-06-11T19:47:18.416Z
22,a3cFU1biFvmBCup9SBvqihpvqVmwCtLsT1mK2dzqwWC94AjJw,Completed,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,40.025560,1.652461,2022-06-11T18:40:54.604Z
17,a3bzJWesF683HSZkvpyXWcejKWVjroyPyF6xGL1GEJBQy5Dh5,Completed,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,6.876467,0.292502,2022-06-11T18:02:54.360Z
29,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,Completed,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,8.842710,0.372572,2022-06-11T17:28:30.592Z
7,a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe,Completed,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,3.733509,0.149104,2022-06-11T16:32:42.406Z
43,a3fVzyUt9RGMLbf3dD5C11GybmJV8mHxKuyKqodhdBbmZuUrY,Completed,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,0.290574,0.011612,2022-06-11T14:06:54.297Z
11,a3btcmyVE6ENtWVyHiX9QnorJfKfA2TsSCF43urDeNAWKueH6,Completed,a3bzFrZ5kXYpaaD5NbapUDSfjZPQTWFKGwSbMmGeRAL8BGrCs,0.142748,0.005905,2022-06-10T17:49:36.422Z


In [66]:
boy = premium_redeems_x.groupby(['vault.accountId']).agg(
    {
        "ksm_premium": sum,
        "btc_requested": sum,
     "request.timestamp": max,
     
    }
).reset_index().rename(columns={ "request.timestamp": "last_action"
                            }, inplace=False)#.sort_values('last_action', ascending=False)
enrich_df(boy, 'vault.accountId')
boy


,vault.accountId,ksm_premium,btc_requested,last_action,label_vault.accountId
0,a3aEG5Y5V5ark2HYLXK9h1rX53MwbmP115kwmKuviVNUyg7tD,0.312620,0.013772,2022-06-06T17:05:06.446Z,Selfish/Vault
1,a3aVQQpfHH49ACiLzDZP8w8eTA2oW7gvifX7E6dLsBJTBdzVR,0.356862,0.017594,2022-05-12T14:07:48.404Z,Lucky Friday/Vault/Self Issuer/K>200/Top KINT ...
2,a3adPv7McJ11sBVApm9ZcR52wpQAKBLFjd2n9E6DH1cBkccMg,0.535421,0.027481,2022-05-11T13:09:26.857Z,Top Issuer/Selfish/Top 20 Vault/Self Issuer/Da...
3,a3addPTx9ngWGKq3dguw7vs7NA2PimcDUHWJ32HsuoFL74zdo,1.894507,0.101199,2022-05-11T19:10:12.682Z,Simon Kraus/@sik/Selfish/Top 20 Vault/Self Iss...
4,a3b5jAhU7CKnBKA3y92HkiCqRTbSKRMK5kN3RVPXcKoH4SDaC,0.132526,0.004944,2022-06-13T01:09:54.238Z,Social/Vault/Top KINT Mover/Top KSM Sink
5,a3bAsW2TQb1c7HrcZd2y81SdCVSGEA5pRkfg576G2EyxQupbw,0.104703,0.004906,2022-05-12T04:59:42.333Z,Selfish/Vault/Top KINT Mover/Top KSM Sink/Top ...
6,a3bBUocXg2cDMctxmEUaQPteH67NyLqxrYwX7soHuZaCHV5nE,0.377534,0.021095,2022-05-11T09:58:30.404Z,Selfish/Vault/Top KINT Mover/Top KSM Mover/Top...
7,a3baaLbC1JMHJLJ2HwEQMz3S5VuiCWBYy4i66Ziq1vXzmVU6b,0.075639,0.003799,2022-05-16T18:13:30.351Z,@spazcoin/chaos DAO/Social/Vault
8,a3bkRMWVb3C3JyXVFvfR5Brq435C6eqMr42YhA759JyKTzRzQ,2.546420,0.170977,2022-05-02T20:03:48.400Z,Top Redeemer/Social/Top 20 Vault/Self Issuer/D...
9,a3bzFrZ5kXYpaaD5NbapUDSfjZPQTWFKGwSbMmGeRAL8BGrCs,2.435458,0.098035,2022-06-13T08:31:24.536Z,@boyswan/Selfish/Vault/Top KINT Mover/Top KINT...


In [67]:
transform = []
for k, v in kts.items():
  d = dict(key = k, labels="/".join(v))
  transform.append(d)

pd.DataFrame(transform)

,key,labels
0,a3eZhSC12zE4D49ir4QkxZwDC3jU6iwNQ953ZZzQsCLTnPJjs,Simon Kraus/@sik/Top 20 Vault/Self Issuer/nPJj...
1,a3fxS24Bduq8cxT4oV7RQ6q6vfEdSEkaapER75BS4rF7U6Q63,Simon Kraus/@sik/Top Redeemer/Selfish/Vault/Da...
2,a3addPTx9ngWGKq3dguw7vs7NA2PimcDUHWJ32HsuoFL74zdo,Simon Kraus/@sik/Selfish/Top 20 Vault/Self Iss...
3,a3bccyaV6tCoqaWqByeqvXo5kBbd1m4yCPKyX937HB1APTt7Y,rodrigo.barrios/hypersphere/Social/Vault
4,a3bzFrZ5kXYpaaD5NbapUDSfjZPQTWFKGwSbMmGeRAL8BGrCs,@boyswan/Selfish/Vault/Top KINT Mover/Top KINT...
...,...,...
2133,0x817066de5c94e7122a231ac3bb3b28ee8f3acaba,Top KBTC Sink
2134,0x74b5f4572a9057580bf5548b6edcafcc72a31814,Top KBTC Sink
2135,0x98ed2a2f546d93b4aaaa77d5504e7ecac49b2a10,Top KBTC Sink
2136,a3cXeDvWBDfrivt3RfQHFJKaoNy1UFoddghbh27kUwBkXJttv,Top KBTC Sink


In [71]:
adhoc = """
query MyQuery {
  transfers(orderBy: timestamp_DESC) {
    amount
    from {
      karura
      kintsugi
      kusama
      moonriver
      id
    }
    fromChain
    timestamp
    to {
      id
      karura
      kintsugi
      kusama
      moonriver
    }
    toChain
    token
  }
}
"""


In [ ]:
kintsugi